Нехай потрібно написати примітивну нейронну мережу, яка обчислює значення виразу 
$$w * x + b$$ 
Це можна було б зробити наступним чином:

In [1]:
import tensorflow as tf


class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(5.0)
        self.b = tf.Variable(5.0)

    def __call__(self, x):
        return self.w * x + self.b


simple_module = SimpleModule(name="simple")
simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

Обов'язковою є реалізація магічного методу `__cal__`, оскільки всі фактичні обчислення відбуваються саме там.

Тепер ми можемо спробувати створити складнішу нейронну мережу. Нехай потрібно створити нейромережу, в якій **2 шари**. На першому шарі має 3 нейрони з трьома входами, на другому - 1, в якості активаційної функції використовуватимемо ReLU. Реалізація такої нейромережі представлена далі.

In [2]:
class DenseLayer(tf.Module):
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name="w")
        self.b = tf.Variable(tf.zeros([out_features]), name="b")

    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)


class NN(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)
    self.layer_1 = DenseLayer(in_features=3, out_features=3)
    self.layer_2 = DenseLayer(in_features=3, out_features=1)

  def __call__(self, x):
    x = self.layer_1(x)
    return self.layer_2(x)


nn = NN(name="neural_network")
print("Results:", nn(tf.constant([[2.0, 2.0, 2.0]])))


Results: tf.Tensor([[0.]], shape=(1, 1), dtype=float32)


Mи створили просту нейронну мережу, що складається із двох шарів. Поки що така нейронна мережа марна. Щоб із нею можна було щось робити, її треба навчити. Навчати нейронні мережі можна по-різному і в tensorflow для цього є відповідні механізми. Для демонстрації нам вистачить класичного градієнтного спуску. Далі ми побачимо, що цей алгоритм досить простий у реалізації за допомогою бібліотеки, яку ми розглядаємо. Нехай потрібно навчити нейронну мережу, що складається з одного нейрона. 

Фактично ми маємо лінійну модель виду:
$$
a(x) = w1x1 + w2x2 + ...+ wnxn + b
$$
Створимо клас для даної моделі:

In [3]:
import tensorflow as tf


class LinearModel(tf.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.w = tf.Variable(5.0)
        self.b = tf.Variable(0.0)

    def __call__(self, x):
        return self.w * x + self.b

In [ ]:
def loss(target_y, predicted_y):
    return tf.reduce_mean(tf.square(target_y - predicted_y))


def train(model, x, y, learning_rate):
    with tf.GradientTape() as t:
        current_loss = loss(y, model(x))
        dw, db = t.gradient(current_loss, [model.w, model.b])
        model.w.assign_sub(learning_rate * dw)
        model.b.assign_sub(learning_rate * db)

Для оновлення ваг ми використовуємо метод `assign_sub`, який веде себе як оператор `-=`. Функція `train` виконають лише одну ітерацію навчання, якої явно недостатньо. Тому нам знадобиться ще одна функція, у якій ми будемо безпосередньо тренувати модель:

In [5]:
def training_loop(model, x, y):
    for epoch in range(10):
        train(model, x, y, learning_rate=0.1)
        current_loss = loss(y, model(x))
        print(f"loss: {current_loss}")

Для того щоб протестувати навчання моделі згенеруємо тестові дані:

In [6]:
TRUE_W = 3.0
TRUE_B = 2.0

NUM_EXAMPLES = 1000

x = tf.random.normal(shape=[NUM_EXAMPLES])
noise = tf.random.normal(shape=[NUM_EXAMPLES])
y = x * TRUE_W + TRUE_B + noise

Тепер зберемо все разом:

In [7]:
linear_model = LinearModel()
training_loop(linear_model, x, y)

loss: 5.9825215339660645
loss: 4.1342597007751465
loss: 2.9626123905181885
loss: 2.2198054790496826
loss: 1.7488261461257935
loss: 1.4501665830612183
loss: 1.2607570886611938
loss: 1.1406190395355225
loss: 1.0644086599349976
loss: 1.0160578489303589
